In [13]:
import apache_beam as beam
with beam.Pipeline() as pipeline:
    plants = (pipeline
             |'Gardening plants' >> beam.Create([
          '🍓Strawberry 🥕Carrot 🍆Eggplant',
          '🍅Tomato 🥔Potato',
      ])
              | 'split words' >> beam.FlatMap(str.split)
              |beam.Map(print)
             )

🍓Strawberry
🥕Carrot
🍆Eggplant
🍅Tomato
🥔Potato


In [16]:
import apache_beam as beam

def split_words(text):
    return text.split(',')

with beam.Pipeline() as p:
    plants = (
                p | 'Gardening plants' >> beam.Create([
          '🍓Strawberry,🥕Carrot,🍆Eggplant',
          '🍅Tomato,🥔Potato',
      ])
        #|beam.FlatMap(split_words)
        |beam.FlatMap(lambda x: x.split(','))
        |beam.Map(print)
    )
    

🍓Strawberry
🥕Carrot
🍆Eggplant
🍅Tomato
🥔Potato


In [18]:
import apache_beam as beam
with beam.Pipeline() as p:
    plants= (p|beam.Create([['🍓Strawberry', '🥕Carrot', '🍆Eggplant'],
          ['🍅Tomato', '🥔Potato'],])
            |beam.FlatMap(lambda x: x)
            |beam.Map(print))

🍓Strawberry
🥕Carrot
🍆Eggplant
🍅Tomato
🥔Potato


In [19]:
import apache_beam as beam
def generate_elements(elements):
    for element in elements:
        yield element
        
with beam.Pipeline() as p:
    plants=(
    p|beam.Create([['🍓Strawberry', '🥕Carrot', '🍆Eggplant'],
          ['🍅Tomato', '🥔Potato'],
        
    ])|beam.FlatMap(generate_elements)
        |beam.Map(print)
    )

🍓Strawberry
🥕Carrot
🍆Eggplant
🍅Tomato
🥔Potato


In [23]:
import apache_beam as beam
def format_plant(icon,plant):
    if icon:
        yield '{}{}'.format(icon,plant)
with beam.Pipeline() as p :
    plants = (p| 'Gardening plants' >> beam.Create([
          ('🍓', 'Strawberry'),
          ('🥕', 'Carrot'),
          ('🍆', 'Eggplant'),
          ('🍅', 'Tomato'),
          ('🥔', 'Potato'),
          (None, 'Invalid'),
      ])
        |beam.FlatMapTuple(format_plant)
        |beam.Map(print)
             )
        

🍓Strawberry
🥕Carrot
🍆Eggplant
🍅Tomato
🥔Potato


In [24]:
import apache_beam as beam
def split_words(text, delimiter=None):
    return text.split(delimiter)
with beam.Pipeline() as pipeline:
    plants = (
    pipeline|beam.Create([
        '🍓Strawberry,🥕Carrot,🍆Eggplant',
          '🍅Tomato,🥔Potato',
    ])
    |beam.FlatMap(split_words,delimiter=',')
    |beam.Map(print)
    )

🍓Strawberry
🥕Carrot
🍆Eggplant
🍅Tomato
🥔Potato


In [25]:
import apache_beam as beam
def normalize_and_validate_durations(plant, valid_durations):
    plant['duration'] = plant['duration'].lower()
    if plant['duration'] in valid_durations:
        yield plant

with beam.Pipeline() as p:
    valid_durations = p | beam.Create([
        'annual',
      'biennial',
      'perennial',
    ])
    valid_plants = (
        p| 'Gardening plants' >> beam.Create([
          {
              'icon': '🍓', 'name': 'Strawberry', 'duration': 'Perennial'
          },
          {
              'icon': '🥕', 'name': 'Carrot', 'duration': 'BIENNIAL'
          },
          {
              'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'
          },
          {
              'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'
          },
          {
              'icon': '🥔', 'name': 'Potato', 'duration': 'unknown'
          },
      ])
        |beam.FlatMap(
        normalize_and_validate_durations,
            valid_durations=beam.pvalue.AsIter(valid_durations)
        )|beam.Map(print)
        )

{'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'}
{'icon': '🥕', 'name': 'Carrot', 'duration': 'biennial'}
{'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'}
{'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'}


In [35]:
import apache_beam as beam
def replace_duration_if_valid(plant, durations):
  if plant['duration'] in durations:
    plant['duration'] = durations[plant['duration']]
    yield plant

with beam.Pipeline() as pipeline:
    durations = pipeline | 'Durations dict' >> beam.Create([
      (0, 'annual'),
      (1, 'biennial'),
      (2, 'perennial'),
  ])
    
    valid_plants = (
      pipeline| 'Gardening plants' >> beam.Create([
          {
              'icon': '🍓', 'name': 'Strawberry', 'duration': 2
          },
          {
              'icon': '🥕', 'name': 'Carrot', 'duration': 1
          },
          {
              'icon': '🍆', 'name': 'Eggplant', 'duration': 2
          },
          {
              'icon': '🍅', 'name': 'Tomato', 'duration': 0
          },
          {
              'icon': '🥔', 'name': 'Potato', 'duration': -1
          },
      ])
        | 'Replace duration if valid' >> beam.FlatMap(
          replace_duration_if_valid,
          durations=beam.pvalue.AsDict(durations),
      )
      | beam.Map(print))

{'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'}
{'icon': '🥕', 'name': 'Carrot', 'duration': 'biennial'}
{'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'}
{'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'}
